In [1]:
 !pip install faiss-cpu sentence-transformers langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1

In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import os
from langchain_google_genai import ChatGoogleGenerativeAI

In [5]:
os.environ["GOOGLE_API_KEY"]="your api key"

In [6]:
sentences=[
    "I love to cook.",
    "AI is a most advanced and growing technology.",
    "Dogs are the good animals.",
    "Nowadays, the education management getting worse.",
    "In medical system, scientists are trying to find a drugs for the disease.",
    "Cats are cute animals.",
    "Wild animals are also good."
]

In [7]:
model=SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
embeddings=model.encode(sentences) #shape (7,384) 7-number of sentences
embeddings=np.array(embeddings).astype("float32")

In [9]:
embeddings.shape

(7, 384)

In [10]:
dimension=embeddings.shape[1]
index=faiss.IndexFlatL2(dimension) #calculate index for dimensions using euclidean distance for giving index for a group
index.add(embeddings)

In [11]:
query="I adore wild animals"
query_vector=model.encode([query]).astype("float32")

top_k=4
distances,indices=index.search(query_vector,top_k)
retrieved_chunks=[]
print("Query:",query)
for i in indices[0]:
  print(sentences[i])
  retrieved_chunks.append(sentences[i])

Query: I adore wild animals
Wild animals are also good.
Dogs are the good animals.
I love to cook.
Cats are cute animals.


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema import HumanMessage

# Create Gemini chat model instance
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

# Prepare context and question
context = "\n".join(retrieved_chunks)
prompt = f"""Answer the question based on the context below:
Context:
{context}
Question: {query}
Answer:"""

# Call the model
response = llm([HumanMessage(content=prompt)])

# Print the final answer
print("\nFinal Answer:\n", response.content)


/tmp/ipython-input-14-1081936374.py:16: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=prompt)])



Final Answer:
 Based on the provided text, the answer is **uncertain**.  The text states "Wild animals are also good," which is a positive statement, but it doesn't explicitly say that the speaker adores them.
